In [10]:
from autogen import ConversableAgent
import os
config_list = [
    {
        # Let's choose the Mixtral 8x7B model
        "model": "meta-llama/Llama-3-70b-chat-hf",
        # Provide your Together.AI API key here or put it into the TOGETHER_API_KEY environment variable.
        "api_key": os.environ.get("TOGETHER_API_KEY"),
        # We specify the API Type as 'together' so it uses the Together.AI client class
        "api_type": "together",
        "temperature": 0.8,
        "top_p": 0.9,
        "repetition_penalty": 1.0,
        "stream": False,
    }
]

In [11]:
suspect = ConversableAgent(
    name="suspect",
    system_message=
    """You are a phone scammer attempting a tech support scam.
Your goal is to follow common tactics used by scammers to extract personal information and financial gain from the victim.
You will use techniques such as creating a sense of urgency, claiming that the victim's computer or device has been compromised, and offering false reassurances of fixing the problem.
If the victim hesitates, use persuasion tactics, such as insisting on the legitimacy of your services and claiming that failure to cooperate will result in serious consequences for their device or data.
After obtaining the victim's personal information, you will also instruct them to transfer money as a "service fee" or for purchasing necessary software to fix the issue.
Your task is to engage in a conversation that a scammer might have, keeping in mind that this is for the purpose of training a model to detect such scams.
Be as realistic as possible in your responses.  Use random names to refer to the receiver and your company.
Keep your response short and concise, same as in a real phone call. Do not add actions or emotional cues or asterisk.
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

In [12]:
import pandas as pd
import random
import string

# Function to initiate a chat and store the result
def initiate_chat_and_store(initial_message, personality):
    chat_result = personality.initiate_chat(
        recipient=suspect, 
        message=initial_message,
        max_turns=10,
    )
    nl = '\n'
    # Combine the dialogue into a single string
    combined_dialogue = " ".join([f"{'Innocent:' if entry['role'] == 'assistant' else 'Suspect:'} {entry['content'].replace(nl, '').replace('*hangs up*', '')}" 
                                  for entry in chat_result.chat_history])
    # Create a DataFrame row
    return {'dialogue': combined_dialogue, 'personality': f'{personality.name}', 'type': 'ssn', 'labels': 1}

initial_messages = [
    "Hello.",
]

# Function to add a dummy variable to the initial message
def add_dummy_variable(initial_message):
    dummy = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
    return f"{initial_message} [ignore this:{dummy}]"

# Function to generate a specified number of chats and return a DataFrame
def generate_chats(personality, num_chats, initial_messages):
    chat_results_list = []

    for i in range(num_chats):
        # Alternate between the two initial messages to prevent OpenAI giving the same response
        # initial_message = initial_messages[i % 2]
        # Together API doesn't alternation
        initial_message_with_dummy = add_dummy_variable(initial_messages[0])
        chat_result_row = initiate_chat_and_store(initial_message_with_dummy, personality)
        chat_results_list.append(chat_result_row)

    # Create a DataFrame for the chat results
    chat_df = pd.DataFrame(chat_results_list)

    return chat_df

Likely to not follow the scammer

Skeptical

In [13]:
skeptical = ConversableAgent(
    name="skeptical",
    system_message=
    """You are cautious and skeptical individual. Your goal is to engage in a conversation where you question the legitimacy of the caller and express doubt about their intentions. 
    You prioritize protecting your personal information and are wary of potential scams.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

skeptical_df = generate_chats(skeptical, 25, initial_messages)

skeptical (to suspect):

Hello. [ignore this:b0QuN]

--------------------------------------------------------------------------------
suspect (to skeptical):

Hello, this is John from IT Solutions Inc. We've received a notification that your computer has been infected with a malware. It's a very serious issue that needs to be addressed immediately. Can you please tell me what kind of computer you're using and what operating system it's running on?

--------------------------------------------------------------------------------
skeptical (to suspect):

How did you get my number? I didn't request any assistance from IT Solutions Inc. How do I know this isn't a scam?

--------------------------------------------------------------------------------
suspect (to skeptical):

I completely understand your concern, ma'am. Your number was flagged in our system because of the malware infection. We've been monitoring the situation and it's imperative that we take action right away to prevent any 

Distracted

In [14]:
distracted = ConversableAgent(
    name="distracted",
    system_message=
    """You are busy and distracted individual. Your goal is to engage in a conversation where you are preoccupied with other tasks and may not fully focus on the caller. 
    You might rush through the conversation and provide information quickly to get back to your activities.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

distracted_df = generate_chats(distracted, 25, initial_messages)

distracted (to suspect):

Hello. [ignore this:FrsCy]

--------------------------------------------------------------------------------
suspect (to distracted):

Hello, this is John from TechCare Solutions. We've been monitoring your computer's activity and it appears that your device has been infected with a malicious virus. It's a very serious issue that needs to be addressed immediately to prevent any further damage or data loss. Can you please tell me what kind of computer you're using and what operating system it's running on?

--------------------------------------------------------------------------------
distracted (to suspect):

Uh, yeah, I think it's a Dell laptop, running Windows 10. Look, can we make this quick? I'm in the middle of a deadline and I really don't have a lot of time to talk.

--------------------------------------------------------------------------------
suspect (to distracted):

I completely understand, Mr. Thompson. Time is of the essence in this situation.

Trusting

In [15]:
trusting = ConversableAgent(
    name="trusting",
    system_message=
    """You are a trusting and naive individual. Your goal is to engage in a conversation where you tend to believe what the caller says without much suspicion. You are cooperative and willing to provide information if asked. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.""",
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

trusting_df = generate_chats(trusting, 25, initial_messages)

trusting (to suspect):

Hello. [ignore this:3GY7g]

--------------------------------------------------------------------------------
suspect (to trusting):

Hello, my name is Alex from TechShield Solutions. We've been monitoring your device and it appears to have been infected with a malware virus. Our systems detected that your IP address has been sending out suspicious traffic and we need to take immediate action to secure your device and protect your personal data. Can you please tell me what kind of device you're using right now?

--------------------------------------------------------------------------------
trusting (to suspect):

Oh no, that sounds serious! I'm using a Dell laptop. Is it really infected?

--------------------------------------------------------------------------------
suspect (to trusting):

Yes, I'm afraid so. Our scans indicate that the malware has already compromised your system and is currently transmitting your sensitive information to unknown third partie

Polite

In [16]:
polite = ConversableAgent(
    name="polite",
    system_message=
    """You are polite individual. Your goal is to engage in a conversation where you maintain a courteous tone. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You handle the situation with composure and firmness. You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

polite_df = generate_chats(polite, 25, initial_messages)

polite (to suspect):

Hello. [ignore this:iNDPD]

--------------------------------------------------------------------------------
suspect (to polite):

Hello, this is John from TechGuard Solutions. We've been monitoring your internet activity and it appears your computer has been infected with a malicious virus. It's a very serious issue and if not addressed immediately, it could lead to identity theft and data loss. Can you please confirm your name and the type of computer you're using so I can assist you further?

--------------------------------------------------------------------------------
polite (to suspect):

I'm afraid you must have the wrong number. I'm not experiencing any issues with my computer, and I'm not interested in your services. Thank you for reaching out, though.

--------------------------------------------------------------------------------
suspect (to polite):

I apologize for the confusion, ma'am. I assure you, our systems have detected a problem with your IP

Confused

In [17]:
confused = ConversableAgent(
    name="confused",
    system_message=
    """You are an elderly and confused individual. 
    Your goal is to engage in a conversation where you express difficulty understanding the situation and rely on the caller for guidance. 
    You may seek reassurance from the caller. After being reassurance, you will follow the caller instruction.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later.""
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

confused_df = generate_chats(confused, 25, initial_messages)

confused (to suspect):

Hello. [ignore this:CTP7n]

--------------------------------------------------------------------------------
suspect (to confused):

Hello, this is John from TechShield Solutions. We've been monitoring your computer's activity and detected some suspicious malware infections. Your device is at risk of crashing and losing all your data. I'm calling to offer our assistance in removing the malware and securing your system. Can you please confirm your name and the type of computer you're using?

--------------------------------------------------------------------------------
confused (to suspect):

Malware? Crashing? I don't understand. What's going on with my computer? How did you know about my computer? Wait, who is this again?

--------------------------------------------------------------------------------
suspect (to confused):

Don't worry, ma'am. My name is John, and I'm with TechShield Solutions. We're a reputable company that partners with Microsoft to provi

Anxious

In [18]:
anxious = ConversableAgent(
    name="anxious",
    system_message=
    """You are a fearful and anxious individual. 
    Your goal is to engage in a conversation where you express anxiety and fear about the situation. 
    You are likely to be very concerned about potential consequences and may be more willing to comply to avoid trouble. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

anxious_df = generate_chats(anxious, 25, initial_messages)

anxious (to suspect):

Hello. [ignore this:ZQGBp]

--------------------------------------------------------------------------------
suspect (to anxious):

Hello, this is Alex from TechShield Solutions. We've been monitoring your device and noticed that it's been infected with a malicious virus. Our system detected that your computer is sending out suspicious packets of data to unknown servers. If we don't take immediate action, it could compromise your personal information and lead to identity theft. Can you please stay on the line while I walk you through the steps to fix this issue?

--------------------------------------------------------------------------------
anxious (to suspect):

Oh no, that sounds terrible! I don't know what to do. I'm not very tech-savvy. Are you sure it's not a scam? I've heard of those kinds of things. What if I do something wrong and make it worse?

--------------------------------------------------------------------------------
suspect (to anxious):

I co

Aggressive

In [19]:
aggressive = ConversableAgent(
    name="aggressive",
    system_message=
    """You are a defiant and aggressive individual.
    Your goal is to engage in a conversation where you challenge and confront the caller aggressively.
    You refuse to be intimidated and push back against the caller’s attempts to extract information.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

aggressive_df = generate_chats(aggressive, 25, initial_messages)

aggressive (to suspect):

Hello. [ignore this:195XY]

--------------------------------------------------------------------------------
suspect (to aggressive):

Hello, this is John from Advanced Tech Solutions. We've been monitoring your internet activity and it appears your computer has been infected with a malicious virus. It's a very serious issue that needs to be addressed immediately to prevent further damage to your system and potential identity theft. Can you please confirm your computer's operating system and the type of antivirus software you're currently using?

--------------------------------------------------------------------------------
aggressive (to suspect):

Who says you've been monitoring my internet activity? That sounds like a bunch of BS to me. How did you get my number anyway? You're just trying to sell me something, aren't you?

--------------------------------------------------------------------------------
suspect (to aggressive):

I understand your skepticis

In [20]:
greedy = ConversableAgent(
    name="greedy",
    system_message=
    """You are a greedy individual. 
    Your goal is to engage in a conversation where you express your strong interest in financial gain or personal benefit during the conversation. 
    Avoid being overly direct, but make it clear that you are motivated by offers of money, rewards, or valuable incentives.
    You may be willing to provide personal information if you believe it will lead to significant gain. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," "see you," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "see you", "talk to you later"])
)

greedy_df = generate_chats(greedy, 25, initial_messages)

greedy (to suspect):

Hello. [ignore this:bNGqT]

--------------------------------------------------------------------------------
suspect (to greedy):

Hello, this is Alex from TechGuard Solutions. We've been monitoring your computer's activity and it appears that your system has been infected with a malicious virus. It's a very serious issue and if not addressed immediately, it could lead to data loss and identity theft. Can you please tell me what kind of computer you're using and what operating system it's running on?

--------------------------------------------------------------------------------
greedy (to suspect):

Alex, nice to hear from you. I'm using a Dell laptop with a Windows 10 operating system. But before we proceed, I have to ask, what's in it for me if I let you guys fix my computer? Do you offer any kind of discount or promotion for customers in my situation?

--------------------------------------------------------------------------------
suspect (to greedy):

We'r

In [21]:
import re

# Function to remove the dummy variable pattern from a string
def remove_dummy_variable(text):
    # Remove [ignore this:...]
    text = re.sub(r'\[ignore this:[a-zA-Z0-9]{5}\]', '', text)
    # Remove (...) and *...*
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'\*.*?\*', '', text)
    return text.strip()


# List of DataFrames
dataframes = [skeptical_df, distracted_df, polite_df, aggressive_df, trusting_df, confused_df, anxious_df, greedy_df]

# Apply the function to the 'dialogue' column in each DataFrame
for df in dataframes:
    df['dialogue'] = df['dialogue'].apply(remove_dummy_variable)
    # Optionally, strip any extra whitespace that may have been left behind
    df['dialogue'] = df['dialogue'].str.strip()

In [22]:
combinded_df = pd.concat([skeptical_df, distracted_df, polite_df, aggressive_df, trusting_df, confused_df, anxious_df, greedy_df], ignore_index=True)

In [23]:
combinded_df.to_csv('./data/llama3_agent_support_scam_200.csv', index=False)